In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import sys
import os
import pyarrow.parquet as pq
import torch

In [2]:
def read_parquet_in_batches(file_path: str, batch_size=100000):

    parquet_file = pq.ParquetFile(file_path)

    total_rows = parquet_file.metadata.num_rows
    processed_rows = 0

    for batch in parquet_file.iter_batches(batch_size=batch_size):
        batch_df = batch.to_pandas()

        processed_rows += len(batch_df)
        progress = (processed_rows / total_rows) * 100
        print(f'Progress: {progress:.2f}%')
        
        yield batch_df

In [3]:
counter = 0
batches = []
for batch in read_parquet_in_batches(file_path='../data/train/train_data_validate.parquet'):
    if not batches:
        batches.append(batch)
    else:
        batches.append(batch)
        break
    
for batch in batches:
    batch.drop(columns=['variantid1', 'variantid2'], inplace=True)

Progress: 8.56%
Progress: 17.12%


In [4]:
batches[0].head()

,embedding1,embedding2,target
0,"[0.5318107604980469, 0.35363996028900146, -0.7...","[0.5318107604980469, 0.35363996028900146, -0.7...",1
1,"[0.4308440089225769, 0.7620932459831238, 0.793...","[0.5668608546257019, 0.9573432803153992, 1.017...",1
2,"[-0.36238163709640503, 0.4316844344139099, -0....","[-0.25123998522758484, 0.3757574260234833, -0....",0
3,"[0.7327960729598999, -0.7488707900047302, 0.55...","[0.7327960729598999, -0.7488707900047302, 0.55...",1
4,"[-1.3140270709991455, -0.8071212768554688, 0.7...","[-0.49589139223098755, -0.5760805606842041, 0....",0


In [5]:
test_tensor = torch.tensor(batches[0].iloc[0, 1])

In [6]:
test_tensor.shape

torch.Size([320])

In [7]:
from torch.utils.data import Dataset, DataLoader

class SiameseDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get the tensors and label from the dataframe
        tensor1 = torch.tensor(self.dataframe.iloc[idx, 0].tolist(), dtype=torch.float32)
        tensor2 = torch.tensor(self.dataframe.iloc[idx, 1].tolist(), dtype=torch.float32)
        label = torch.tensor(self.dataframe.iloc[idx, 2].tolist(), dtype=torch.float32)
        
        return tensor1, tensor2, label

# Load your dataframe
# dataframe = pd.read_csv('../data/train/siamence_main_pic.parquet')
train_dataframe = batches[0]
test_dataframe = batches[1]

# Initialize the dataset and dataloader
train_dataset = SiameseDataset(train_dataframe)
test_dataset = SiameseDataset(test_dataframe)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class SiamenceNetwork(nn.Module):
    def __init__(self):
        super(SiamenceNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(320, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
        
    def forward_one(self, x):
        return self.fc(x)
    
    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

In [9]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        # Compute Euclidean distance
        euclidean_distance = F.pairwise_distance(output1, output2)
        
        # Contrastive loss
        loss = torch.mean(
            (label) * torch.pow(euclidean_distance, 2) +  # Similar pairs: distance squared
            (1 - label) * torch.pow(F.relu(self.margin - euclidean_distance), 2)  # Dissimilar pairs: margin - distance squared
        )
        return loss
        

In [10]:
# Initialize model, loss, and optimizer

# Check if GPU is available and use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device - {device}')

# Initialize model, loss, and optimizer
model = SiamenceNetwork().to(device)
criterion = ContrastiveLoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device - cpu


In [11]:
def train_model(model, dataloader, criterion, optimizer, device, num_epochs, CE=False):
    for epoch in range(num_epochs):
        epoch_loss = 0.0  # To keep track of epoch loss
        for tensor1, tensor2, label in dataloader:
            # Move data to GPU
            tensor1 = tensor1.to(device)
            tensor2 = tensor2.to(device)
            label = label.to(device)

            optimizer.zero_grad()

            # Forward pass
            if CE:
                output = model(tensor1, tensor2)
                loss = criterion(output, label.unsqueeze(1))
            else:
                output1, output2 = model(tensor1, tensor2)
                loss = criterion(output1, output2, label)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * tensor1.size(0)  # Accumulate loss

        # Average loss for the epoch
        epoch_loss /= len(dataloader.dataset)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

In [ ]:
num_epochs = 4
train_model(model, train_dataloader, criterion, optimizer, device, num_epochs)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score

def test_model_with_accuracy(model, dataloader, criterion, device, threshold=0.5, CE=False):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    num_samples = 0
    true_values = []
    predicted_values = []
    
    with torch.no_grad():  # Disable gradient calculation
        for tensor1, tensor2, label in dataloader:
            tensor1 = tensor1.to(device)
            tensor2 = tensor2.to(device)
            label = label.to(device)

            # Forward pass
            if CE:
                output = model(tensor1, tensor2)
                loss = criterion(output, label.unsqueeze(1))
                predictions = (output < threshold).float()
            else:
                output1, output2 = model(tensor1, tensor2)
                loss = criterion(output1, output2, label)
                euclidean_distance = F.pairwise_distance(output1, output2)
                predictions = (euclidean_distance < threshold).float()
            
            total_loss += loss.item() * tensor1.size(0)  # Accumulate loss
            num_samples += tensor1.size(0)
            
            # Move to CPU and detach before appending to lists
            predicted_values.extend(predictions.cpu().detach().numpy())
            true_values.extend(label.cpu().detach().numpy())
            
    
    true_values = np.array(true_values)
    predicted_values = np.array(predicted_values)
    
    average_loss = total_loss / num_samples
    accuracy = accuracy_score(true_values, predicted_values)
    recall = recall_score(true_values, predicted_values)
    f1 = f1_score(true_values, predicted_values)
    return average_loss, accuracy, recall, f1


In [ ]:
thresholds = []
losses = []
accuracies = []
recalls = []
f1s = []

threshold = 0
for i in range(3, 8):
    threshold = i*0.1
    test_loss, test_accuracy, test_recall, test_f1 = test_model_with_accuracy(model, test_dataloader, criterion, device, threshold=threshold)
    thresholds.append(threshold)
    losses.append(test_loss)
    accuracies.append(test_accuracy)
    recalls.append(test_recall)
    f1s.append(f1s)
    print(f'Threshold: {threshold:.4f}, Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, Recall: {test_recall:.4f}, f1: {test_f1:.4f}, ')

In [12]:
class NewNetwork(nn.Module):
    def __init__(self):
        super(NewNetwork, self).__init__()
        # Use the pre-trained fully connected layers
        self.siamese_nn = SiamenceNetwork()
        
        # New layers after concatenation
        self.concat_fc = nn.Sequential(
            nn.Linear(32, 32),  # Keeping the same size initially
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 16),  # Reducing the size gradually
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(16, 8),  # Further reducing the size
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(8, 1),
            nn.Sigmoid()  # Sigmoid activation for binary classification
        )   

    def forward(self, input1, input2):
        # Get outputs from the pretrained fc layers
        output1, output2 = self.siamese_nn(input1, input2)
        
        # Concatenate the two outputs along the feature dimension
        concat_output = torch.cat((output1, output2), dim=1)
        
        # Pass the concatenated output through the new layers
        final_output = self.concat_fc(concat_output)
        
        return final_output

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

new_model = NewNetwork()
new_model = new_model.to(device)

# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(new_model.parameters(), lr=0.001)  # Adam optimizer

# # Optionally freeze the pretrained layers
# for param in new_model.fc.parameters():
#     param.requires_grad = False

In [14]:
num_epochs=10
train_model(new_model, train_dataloader, criterion, optimizer, device, num_epochs, CE=True)

Epoch [1/10], Loss: 0.6437


KeyboardInterrupt: 

In [ ]:
thresholds = []
losses = []
accuracies = []
recalls = []
f1s = []

threshold = 0
for i in range(3, 8):
    threshold = i*0.1
    test_loss, test_accuracy, test_recall, test_f1 = test_model_with_accuracy(new_model, test_dataloader, criterion, device, threshold=threshold, CE=True)
    thresholds.append(threshold)
    losses.append(test_loss)
    accuracies.append(test_accuracy)
    recalls.append(test_recall)
    f1s.append(f1s)
    print(f'Threshold: {threshold:.4f}, Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, Recall: {test_recall:.4f}, f1: {test_f1:.4f}, ')

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10,6))
# plt.plot(thresholds, f1s, label='F1 score', color='blue')
# plt.plot(thresholds, accuracies, label='Accuracy', color='red')
# plt.plot(thresholds, recalls, label='Recall', color='greed')

# plt.xlabel('Threshold')
# plt.ylabel('Score')
# plt.legend()
# plt.grid(True)
# plt.show()